## Step 1: Ingest Data

We will use RESTful API to retrieve traffic camera data from the public City of Calgary website https://data.calgary.ca/Transportation-Transit/Traffic-Cameras/k7p9-kppz/about_data

We will use the python library to help make our requests.

In [1]:
import requests
import pandas as pd
import io

api_url = 'https://data.calgary.ca/resource/k7p9-kppz.csv'


response = requests.get(api_url)

if response.status_code == 200:
  data = io.StringIO(response.text)
  df = pd.read_csv(data)
  display(df.head())
else:
  print("Failed to get data")

,camera_url,quadrant,camera_location,point
0,Camera 87 (http://trafficcam.calgary.ca/loc86....,SE,Stoney Trail / Deerfoot Trail SE,POINT (-113.9766063 50.9007257)
1,Camera 141 (http://trafficcam.calgary.ca/loc14...,SW,17 Avenue / 69 Street SW,POINT (-114.1873508 51.0380985)
2,Camera 4 (http://trafficcam.calgary.ca/loc3.jpg),NE,Memorial Drive / 52 Street E,POINT (-113.9558182 51.053253)
3,Camera 38 (https://trafficcam.calgary.ca/loc37...,NW,Crowchild Trail / Shaganappi Trail NW,POINT (-114.149379 51.0988494)
4,Camera 127 (http://trafficcam.calgary.ca/loc12...,NW,Crowchild Trail / Sarcee Trail NW,POINT (-114.1782042 51.1112551)


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   camera_url       173 non-null    object
 1   quadrant         173 non-null    object
 2   camera_location  173 non-null    object
 3   point            173 non-null    object
dtypes: object(4)
memory usage: 5.5+ KB


In [3]:
df[['Camera_Number', 'URL']] = df['camera_url'].str.extract(r'Camera (\d+) \((http[^\)]+)\)')
df[['longitude', 'latitude']] = df['point'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)', expand=True)
df['latitude'] = pd.to_numeric(df['latitude'])
df['longitude'] = pd.to_numeric(df['longitude'])
df.drop(columns=['camera_url','point'], inplace = True)
display(df.head())

,quadrant,camera_location,Camera_Number,URL,longitude,latitude
0,SE,Stoney Trail / Deerfoot Trail SE,87,http://trafficcam.calgary.ca/loc86.jpg,-113.976606,50.900726
1,SW,17 Avenue / 69 Street SW,141,http://trafficcam.calgary.ca/loc140.jpg,-114.187351,51.038098
2,NE,Memorial Drive / 52 Street E,4,http://trafficcam.calgary.ca/loc3.jpg,-113.955818,51.053253
3,NW,Crowchild Trail / Shaganappi Trail NW,38,https://trafficcam.calgary.ca/loc37.jpg,-114.149379,51.098849
4,NW,Crowchild Trail / Sarcee Trail NW,127,http://trafficcam.calgary.ca/loc126.jpg,-114.178204,51.111255


In [4]:
from IPython.display import Image, display


image_url = 'http://trafficcam.calgary.ca/loc29.jpg'

display(Image(url=image_url))


In [5]:
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


In [11]:
import sys

!git clone https://github.com/ultralytics/yolov5 --tag 6.2  # clone
!mv /content/6.2 /content/yolov5
%pip install -qr /content/yolov5/requirements.txt  # install
sys.path.insert(0,'/content/yolov5')

import torch
import utils
display = utils.notebook_init()  # checks

fatal: Too many arguments.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --[no-]verbose    be more verbose
    -q, --[no-]quiet      be more quiet
    --[no-]progress       force progress reporting
    --[no-]reject-shallow don't clone shallow repository
    -n, --no-checkout     don't create a checkout
    --checkout            opposite of --no-checkout
    --[no-]bare           create a bare repository
    --[no-]mirror         create a mirror repository (implies bare)
    -l, --[no-]local      to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    --hardlinks           opposite of --no-hardlinks
    -s, --[no-]shared     setup as shared repository
    --[no-]recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --[no-]recursive ...  alias of --recurse-submodules
    -j, --[no-]jobs <n>   number of submodules cloned in parallel
    --[no-]template <template-directory>
              

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


AttributeError: module 'utils' has no attribute 'notebook_init'

In [9]:
import urllib.request
import shutil
import os
from IPython.display import Image, display

#
image_url = df.loc[5, 'URL']
image_name = "image_0.jpg"

# Directories setup
images_dir = "/content/images/"
yolo_output_dir = "/content/yolov5/runs/detect/"

# Ensure the directories exist
os.makedirs(images_dir, exist_ok=True)
os.makedirs(yolo_output_dir, exist_ok=True)

# Clear previous YOLO runs to avoid clutter
!rm -rf {yolo_output_dir}*

# Local file path where the image will be saved
local_img_file = os.path.join(images_dir, image_name)

# Download and save the image
with urllib.request.urlopen(image_url) as response, open(local_img_file, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

# Run YOLOv5 to classify the downloaded image
!python /content/yolov5/detect.py --weights yolov5s.pt --img 1024 \
  --conf 0.25 --source {images_dir}

# Since YOLOv5 increments the output folder for each run, find the latest output directory
latest_run_dir = sorted([os.path.join(yolo_output_dir, d) for d in os.listdir(yolo_output_dir) if os.path.isdir(os.path.join(yolo_output_dir, d))])[-1]


detected_image_path = os.path.join(latest_run_dir, image_name)

display(Image(filename=detected_image_path, width=600))


'rm' is not recognized as an internal or external command,
operable program or batch file.
C:\Users\hurdg\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe: can't open file 'c:\\content\\yolov5\\detect.py': [Errno 2] No such file or directory


IndexError: list index out of range

Sets up the directories for storing images and YOLOv5 outputs.
Clears the YOLOv5 detect output directory to avoid clutter from previous runs.
Downloads the specified image from your DataFrame and saves it locally.
Runs the YOLOv5 model on the saved image to perform object detection.
Finds the latest output directory created by YOLOv5 (since YOLOv5 creates a new directory for each run) and constructs the path to the detected image.
Displays the image with detected objects using IPython's Image display function.

In [8]:
import torch

yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

results = yolo_model(local_img_file)

dt = results.pandas().xyxy[0]
dt

c:\Users\hurdg\Documents\MDSA\DATA 608\project\data608\.venv\Lib\site-packages\torch\hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\hurdg/.cache\torch\hub\master.zip


ModuleNotFoundError: No module named 'cv2'

In [10]:
dt2 = dt[['name', 'class']].groupby(by=["name"]).count().reset_index()
dt2.columns=['name', 'count']
dt2

NameError: name 'dt' is not defined